<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/BreastCancerPredictionNeuralNetworksipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the dependencies

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Device Configuration

In [2]:
#check for CUDA availability
device =torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {device}")

Using Device: cuda


Data Collection and PreProcessing

In [3]:
#load the breast cancer dataset
data=load_breast_cancer()
X=data.data
Y=data.target

In [4]:
print(X)
print(Y)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 

In [5]:
print(Y[:5])
#last five

[0 0 0 0 0]


In [6]:
#split the data
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=2)

In [7]:
print(X.shape,X_train.shape,X_test.shape)

(569, 30) (455, 30) (114, 30)


In [8]:
#Standardizing the data
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [9]:
type(X_train)

numpy.ndarray

In [10]:
#convert data to pytorch tensors
X_train=torch.tensor(X_train,dtype=torch.float32).to(device)
Y_train=torch.tensor(Y_train,dtype=torch.float32).to(device)
X_test=torch.tensor(X_test,dtype=torch.float32).to(device)
Y_test=torch.tensor(Y_test,dtype=torch.float32).to(device)

Neural Network Architecture

In [20]:
#define the neural network architecture
class NeuralNet(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(NeuralNet,self).__init__()
    self.fc1=nn.Linear(input_size,hidden_size)
    self.relu=nn.ReLU()
    self.fc2=nn.Linear(hidden_size,output_size)
    self.sigmoid=nn.Sigmoid()

  def forward(self,x):
    out=self.fc1(x)
    out=self.relu(out)
    out=self.fc2(out)
    out=self.sigmoid(out)
    return out

In [21]:
#Intialize the hyper parameters
input_size=X_train.shape[1]
hidden_size=64
output_size=1
learning_rate=0.001
epochs=100

In [22]:
#initialize the neural
model=NeuralNet(input_size,hidden_size,output_size).to(device)

In [23]:
#define laws and criteria
criterian=nn.BCELoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

Training the neural network

In [25]:
for epoch in range(epochs):
  model.train()
  optimizer.zero_grad()
  outputs=model(X_train)
  loss=criterian(outputs,Y_train.view(-1,1))
  loss.backward()
  optimizer.step()


  #calculate accuracy
  with torch.no_grad():
    predicted=outputs.round()
    correct=(predicted==Y_train.view(-1,1)).float().sum()
    accuracy=correct/Y_train.size(0)
  if(epoch+1)%10==0:
    print(f"Epoch [{epoch+1}/{epochs}],loss:{loss.item():.4f},Accuracy:{accuracy.item()*100:.2f}")

Epoch [10/100],loss:0.0985,Accuracy:97.80
Epoch [20/100],loss:0.0821,Accuracy:98.68
Epoch [30/100],loss:0.0713,Accuracy:98.68
Epoch [40/100],loss:0.0637,Accuracy:98.68
Epoch [50/100],loss:0.0580,Accuracy:98.68
Epoch [60/100],loss:0.0536,Accuracy:98.68
Epoch [70/100],loss:0.0501,Accuracy:98.68
Epoch [80/100],loss:0.0470,Accuracy:98.68
Epoch [90/100],loss:0.0443,Accuracy:98.68
Epoch [100/100],loss:0.0419,Accuracy:98.68


Model Evaluation

In [26]:
#evaluate on training data
model.eval()
with torch.no_grad():
  outputs=model(X_train)
  predicted=outputs.round()
  correct=(predicted==Y_train.view(-1,1)).float().sum()
  accuracy=correct/Y_train.size(0)
  print(f"Training Accuracy:{accuracy.item()*100:.2f}")

Training Accuracy:98.90


In [27]:
#evaluate on training data
model.eval()
with torch.no_grad():
  outputs=model(X_test)
  predicted=outputs.round()
  correct=(predicted==Y_test.view(-1,1)).float().sum()
  accuracy=correct/Y_test.size(0)
  print(f"Testing Accuracy:{accuracy.item()*100:.2f}")

Testing Accuracy:95.61
